# Study 2

In [1]:
from sklearn.datasets import make_blobs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn import metrics
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [2]:
data = pd.read_csv("data_study2.csv")
data

,TI,EES,JIMS,SPOS,BFI,ACS,NCS,MOAQ,GSOI,BIO1,...,BIO4,BIO5,BIO6,BIO7,BIO8,BIO9_1,BIO10,BIO11,BIO12,BIO13
0,0,3.666667,3.1,2.500000,2.666667,3.000000,3.000,3.000000,4,4,...,2,3,3,2,3,3.11,5,3,3,3
1,0,4.000000,1.8,5.000000,4.833333,2.888889,1.250,5.000000,1,5,...,2,5,1,2,1,3.53,5,5,5,5
2,0,3.666667,2.9,4.333333,3.833333,3.555556,3.500,5.000000,4,3,...,2,3,2,1,3,3.00,3,5,5,5
3,1,3.666667,3.1,3.166667,3.166667,2.555556,3.125,3.000000,3,3,...,2,4,2,3,3,2.50,3,2,3,5
4,0,3.666667,3.3,4.166667,4.166667,2.777778,3.000,3.666667,5,1,...,1,3,1,1,1,3.93,5,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0,2.000000,2.3,1.333333,3.833333,3.222222,2.750,4.000000,3,1,...,2,4,1,1,1,2.99,3,3,1,5
237,0,4.000000,2.9,3.833333,3.833333,2.888889,2.250,4.000000,3,1,...,2,5,2,4,5,3.74,6,3,3,5
238,0,4.000000,2.4,5.000000,3.000000,3.555556,2.500,4.666667,7,3,...,3,5,2,1,2,3.00,3,3,3,5
239,0,3.000000,2.1,2.333333,1.833333,2.777778,1.875,3.666667,4,4,...,2,5,2,5,1,4.00,8,3,2,5


# Baseline models: biodata only

In [18]:
from sklearn.model_selection import train_test_split
y=data['MOAQ']
X=data.iloc[:,9:22]
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)



In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
#X_train = np.array(X_train).reshape(-1,1)
#y_train = np.array(y_train).reshape(-1,1)

#X_test = np.array(X_test).reshape(-1,1)
#y_test = np.array(y_test).reshape(-1,1)
# Initialize and fit the linear regression model
reg = LinearRegression().fit(X_train, y_train)

# Make predictions on the test set
y_pred_lin = reg.predict(X_test)

# Computing RMSE
rmse_reg = np.sqrt(mean_squared_error(y_test, y_pred_lin))
print(f"RMSE: {rmse}")

RMSE: 0.6613507621106413


In [20]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': range(1, 5),
    'learning_rate': [0.005, 0.05, 0.5]
}

# Initialize GridSearchCV with XGBoost Regressor
grid = GridSearchCV(xgb.XGBRegressor(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='neg_mean_squared_error')

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset
y_pred_xgb = grid.predict(X_test)

# Computing RMSE
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f"RMSE: {rmse_xgb}")



Fitting 10 folds for each of 12 candidates, totalling 120 fits
RMSE: 0.5908881776087875


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
import numpy as np

# Define a simple neural network model for regression
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer with ReLU activation
    Dense(64, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1)  # Output layer for regression (no activation function)
])

# Compile the model with optimizer and loss function suitable for regression
model.compile(optimizer='adam', 
              loss='mean_squared_error',  # Change the loss function to MSE for regression
              metrics=['mean_squared_error'])  # Change the metric to MSE

# Train the model on the training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
y_pred_nn = model.predict(X_test)

# Computing RMSE
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))
print(f"RMSE: {rmse_nn}")

Epoch 1/10


C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8.3465 - mean_squared_error: 8.3465    
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.6665 - mean_squared_error: 1.6665 
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.4349 - mean_squared_error: 1.4349 
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8388 - mean_squared_error: 0.8388 
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9355 - mean_squared_error: 0.9355 
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6741 - mean_squared_error: 0.6741 
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7093 - mean_squared_error: 0.7093 
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5056 - mean_squared_error: 0.5056 
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6098 - mean_squared_error: 0.6098 
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4908 - mean_squared_error: 0.4908 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
RMSE: 0.5689883491181758


In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the pipeline with SVR instead of LinearSVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # It's common to scale inputs for SVM
    ('svr', SVR())  # Use SVR for regression
])

# Define a parameter grid adapted for SVR
param_grid = {
    'svr__kernel': ['linear', 'poly', 'rbf'],  # SVR kernels
    'svr__C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'svr__epsilon': [0.001, 0.01, 0.1, 1]  # Epsilon in the epsilon-SVR model
}

# Initialize RandomizedSearchCV with the SVR estimator and the parameter grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10, n_jobs=-1)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
pred_svr = grid.predict(X_test)

# Computing RMSE
rmse_svr = np.sqrt(mean_squared_error(y_test, pred_svr))
print(f"RMSE: {rmse_svr}")

Fitting 10 folds for each of 10 candidates, totalling 100 fits
RMSE: 0.5452796386682599


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Adjusting the parameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

# Initialize GridSearchCV with the RandomForestRegressor and the parameter grid
grid = GridSearchCV(RandomForestRegressor(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='neg_mean_squared_error')

# Fit to the training data
grid.fit(X_train, y_train)

# Make predictions on the testing dataset
y_pred_rf = grid.predict(X_test)

# Computing RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f"RMSE: {rmse_rf}")

Fitting 10 folds for each of 16 candidates, totalling 160 fits
RMSE: 0.5785559877305311


In [25]:
# Creating a dataset with the model names and their corresponding AUC scores
data = {
    "Model": ["Linear Regression", "XGBoost", "Neural Network", "SVM", "Random Forest"],
    "RMSE": [rmse_reg, rmse_xgb, rmse_nn, rmse_svr, rmse_rf]
}

df = pd.DataFrame(data)

# Writing the dataset to a .csv file
csv_file_path = "base_model_rmse_scores.csv"  # Specify your desired path and filename
df.to_csv(csv_file_path, index=False)

# WRC Model: work-related psychological constructs only

In [30]:
data = pd.read_csv("data_study2.csv")
data

,TI,EES,JIMS,SPOS,BFI,ACS,NCS,MOAQ,GSOI,BIO1,...,BIO4,BIO5,BIO6,BIO7,BIO8,BIO9_1,BIO10,BIO11,BIO12,BIO13
0,0,3.666667,3.1,2.500000,2.666667,3.000000,3.000,3.000000,4,4,...,2,3,3,2,3,3.11,5,3,3,3
1,0,4.000000,1.8,5.000000,4.833333,2.888889,1.250,5.000000,1,5,...,2,5,1,2,1,3.53,5,5,5,5
2,0,3.666667,2.9,4.333333,3.833333,3.555556,3.500,5.000000,4,3,...,2,3,2,1,3,3.00,3,5,5,5
3,1,3.666667,3.1,3.166667,3.166667,2.555556,3.125,3.000000,3,3,...,2,4,2,3,3,2.50,3,2,3,5
4,0,3.666667,3.3,4.166667,4.166667,2.777778,3.000,3.666667,5,1,...,1,3,1,1,1,3.93,5,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0,2.000000,2.3,1.333333,3.833333,3.222222,2.750,4.000000,3,1,...,2,4,1,1,1,2.99,3,3,1,5
237,0,4.000000,2.9,3.833333,3.833333,2.888889,2.250,4.000000,3,1,...,2,5,2,4,5,3.74,6,3,3,5
238,0,4.000000,2.4,5.000000,3.000000,3.555556,2.500,4.666667,7,3,...,3,5,2,1,2,3.00,3,3,3,5
239,0,3.000000,2.1,2.333333,1.833333,2.777778,1.875,3.666667,4,4,...,2,5,2,5,1,4.00,8,3,2,5


In [31]:
from sklearn.model_selection import train_test_split
y=data['MOAQ']
X=data[['EES', 'JIMS', 'SPOS', 'BFI', 'ACS', 'NCS', 'MOAQ', 'GSOI']]
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
#X_train = np.array(X_train).reshape(-1,1)
#y_train = np.array(y_train).reshape(-1,1)

#X_test = np.array(X_test).reshape(-1,1)
#y_test = np.array(y_test).reshape(-1,1)
# Initialize and fit the linear regression model
reg = LinearRegression().fit(X_train, y_train)

# Make predictions on the test set
y_pred_lin = reg.predict(X_test)

# Computing RMSE
rmse_reg = np.sqrt(mean_squared_error(y_test, y_pred_lin))
print(f"RMSE: {rmse}")

RMSE: 0.6613507621106413


In [35]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': range(1, 5),
    'learning_rate': [0.005, 0.05, 0.5]
}

# Initialize GridSearchCV with XGBoost Regressor
grid = GridSearchCV(xgb.XGBRegressor(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='neg_mean_squared_error')

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset
y_pred_xgb = grid.predict(X_test)

# Computing RMSE
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f"RMSE: {rmse_xgb}")

Fitting 10 folds for each of 12 candidates, totalling 120 fits
RMSE: 0.005638880915754558


In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
import numpy as np

# Define a simple neural network model for regression
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer with ReLU activation
    Dense(64, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1)  # Output layer for regression (no activation function)
])

# Compile the model with optimizer and loss function suitable for regression
model.compile(optimizer='adam', 
              loss='mean_squared_error',  # Change the loss function to MSE for regression
              metrics=['mean_squared_error'])  # Change the metric to MSE

# Train the model on the training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
y_pred_nn = model.predict(X_test)

# Computing RMSE
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))
print(f"RMSE: {rmse_nn}")

Epoch 1/10


C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.1668 - mean_squared_error: 6.1668    
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.3794 - mean_squared_error: 1.3794 
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7892 - mean_squared_error: 0.7892 
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6156 - mean_squared_error: 0.6156 
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4251 - mean_squared_error: 0.4251 
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3530 - mean_squared_error: 0.3530 
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2299 - mean_squared_error: 0.2299 
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1934 - mean_squared_error: 0.1934 
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1460 - mean_squared_error: 0.1460 
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1053 - mean_squared_error: 0.1053 
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/stepWARNING:tensorflow:6 out of the last 6 c

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the pipeline with SVR instead of LinearSVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # It's common to scale inputs for SVM
    ('svr', SVR())  # Use SVR for regression
])

# Define a parameter grid adapted for SVR
param_grid = {
    'svr__kernel': ['linear', 'poly', 'rbf'],  # SVR kernels
    'svr__C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'svr__epsilon': [0.001, 0.01, 0.1, 1]  # Epsilon in the epsilon-SVR model
}

# Initialize RandomizedSearchCV with the SVR estimator and the parameter grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10, n_jobs=-1)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
pred_svr = grid.predict(X_test)

# Computing RMSE
rmse_svr = np.sqrt(mean_squared_error(y_test, pred_svr))
print(f"RMSE: {rmse_svr}")

Fitting 10 folds for each of 10 candidates, totalling 100 fits
RMSE: 0.0484159817169608


In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Adjusting the parameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

# Initialize GridSearchCV with the RandomForestRegressor and the parameter grid
grid = GridSearchCV(RandomForestRegressor(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='neg_mean_squared_error')

# Fit to the training data
grid.fit(X_train, y_train)

# Make predictions on the testing dataset
y_pred_rf = grid.predict(X_test)

# Computing RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f"RMSE: {rmse_rf}")

Fitting 10 folds for each of 16 candidates, totalling 160 fits
RMSE: 0.007493966623847667


In [40]:
# Creating a dataset with the model names and their corresponding AUC scores
data = {
    "Model": ["Logistic Regression", "XGBoost", "Neural Network", "SVM", "Random Forest"],
    "RMSE": [rmse_reg, rmse_xgb, rmse_nn, rmse_svr, rmse_rf]
}

df = pd.DataFrame(data)

# Writing the dataset to a .csv file
csv_file_path = "addedWRC_model_rmse_scores.csv"  # Specify your desired path and filename
df.to_csv(csv_file_path, index=False)

# All-variables model

In [3]:
data = pd.read_csv("data_study2.csv")
data

,TI,EES,JIMS,SPOS,BFI,ACS,NCS,MOAQ,GSOI,BIO1,...,BIO4,BIO5,BIO6,BIO7,BIO8,BIO9_1,BIO10,BIO11,BIO12,BIO13
0,0,3.666667,3.1,2.500000,2.666667,3.000000,3.000,3.000000,4,4,...,2,3,3,2,3,3.11,5,3,3,3
1,0,4.000000,1.8,5.000000,4.833333,2.888889,1.250,5.000000,1,5,...,2,5,1,2,1,3.53,5,5,5,5
2,0,3.666667,2.9,4.333333,3.833333,3.555556,3.500,5.000000,4,3,...,2,3,2,1,3,3.00,3,5,5,5
3,1,3.666667,3.1,3.166667,3.166667,2.555556,3.125,3.000000,3,3,...,2,4,2,3,3,2.50,3,2,3,5
4,0,3.666667,3.3,4.166667,4.166667,2.777778,3.000,3.666667,5,1,...,1,3,1,1,1,3.93,5,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0,2.000000,2.3,1.333333,3.833333,3.222222,2.750,4.000000,3,1,...,2,4,1,1,1,2.99,3,3,1,5
237,0,4.000000,2.9,3.833333,3.833333,2.888889,2.250,4.000000,3,1,...,2,5,2,4,5,3.74,6,3,3,5
238,0,4.000000,2.4,5.000000,3.000000,3.555556,2.500,4.666667,7,3,...,3,5,2,1,2,3.00,3,3,3,5
239,0,3.000000,2.1,2.333333,1.833333,2.777778,1.875,3.666667,4,4,...,2,5,2,5,1,4.00,8,3,2,5


In [4]:
from sklearn.model_selection import train_test_split
y=data['MOAQ']
X=data.drop('MOAQ', axis=1)
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
#X_train = np.array(X_train).reshape(-1,1)
#y_train = np.array(y_train).reshape(-1,1)

#X_test = np.array(X_test).reshape(-1,1)
#y_test = np.array(y_test).reshape(-1,1)
# Initialize and fit the linear regression model
reg = LinearRegression().fit(X_train, y_train)

# Make predictions on the test set
y_pred_lin = reg.predict(X_test)

# Computing RMSE
rmse_reg = np.sqrt(mean_squared_error(y_test, y_pred_lin))
print(f"RMSE: {rmse_reg}")

# Extract coefficients
coefficients = reg.coef_
# If X_train is a DataFrame, we can use column names directly
feature_names = X_train.columns

# Create a DataFrame to view the coefficients and feature names
coeff_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients.flatten()})

# Sort the DataFrame by the absolute values of the coefficients to see the most important features
coeff_df = coeff_df.reindex(coeff_df.Coefficient.abs().sort_values(ascending=False).index)
print(coeff_df)
coeff_df.to_csv("lm_coeff_importance.csv", index=False)

RMSE: 0.44867674298929133
   Feature  Coefficient
3     SPOS     0.355633
0       TI    -0.327215
5      ACS     0.305492
16  BIO9_1     0.184857
1      EES     0.171100
11    BIO4     0.087171
9     BIO2     0.087111
4      BFI    -0.085561
18   BIO11     0.048370
20   BIO13     0.045576
10    BIO3    -0.044642
6      NCS    -0.035523
12    BIO5     0.032321
14    BIO7     0.031316
15    BIO8    -0.029762
8     BIO1    -0.028691
13    BIO6     0.019139
2     JIMS     0.015946
7     GSOI     0.014350
17   BIO10     0.011308
19   BIO12    -0.000810


In [47]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': range(1, 5),
    'learning_rate': [0.005, 0.05, 0.5]
}

# Initialize GridSearchCV with XGBoost Regressor
grid = GridSearchCV(xgb.XGBRegressor(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='neg_mean_squared_error')

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset
y_pred_xgb = grid.predict(X_test)

# Computing RMSE
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f"RMSE: {rmse_xgb}")


Fitting 10 folds for each of 12 candidates, totalling 120 fits
RMSE: 0.41763701690038324


In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
import numpy as np

# Define a simple neural network model for regression
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer with ReLU activation
    Dense(64, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1)  # Output layer for regression (no activation function)
])

# Compile the model with optimizer and loss function suitable for regression
model.compile(optimizer='adam', 
              loss='mean_squared_error',  # Change the loss function to MSE for regression
              metrics=['mean_squared_error'])  # Change the metric to MSE

# Train the model on the training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
y_pred_nn = model.predict(X_test)

# Computing RMSE
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))
print(f"RMSE: {rmse_nn}")

Epoch 1/10


C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 14.4416 - mean_squared_error: 14.4416  
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.6911 - mean_squared_error: 1.6911 
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.0373 - mean_squared_error: 2.0373 
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5697 - mean_squared_error: 0.5697 
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8606 - mean_squared_error: 0.8606 
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5423 - mean_squared_error: 0.5423 
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4351 - mean_squared_error: 0.4351 
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3752 - mean_squared_error: 0.3752 
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3839 - mean_squared_error: 0.3839 
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2911 - mean_squared_error: 0.2911 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
RMSE: 0.4381824036482006


In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the pipeline with SVR instead of LinearSVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # It's common to scale inputs for SVM
    ('svr', SVR())  # Use SVR for regression
])

# Define a parameter grid adapted for SVR
param_grid = {
    'svr__kernel': ['linear', 'poly', 'rbf'],  # SVR kernels
    'svr__C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'svr__epsilon': [0.001, 0.01, 0.1, 1]  # Epsilon in the epsilon-SVR model
}

# Initialize RandomizedSearchCV with the SVR estimator and the parameter grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10, n_jobs=-1)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
pred_svr = grid.predict(X_test)

# Computing RMSE
rmse_svr = np.sqrt(mean_squared_error(y_test, pred_svr))
print(f"RMSE: {rmse_svr}")

Fitting 10 folds for each of 10 candidates, totalling 100 fits
RMSE: 0.4543032747468956


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Adjusting the parameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

# Initialize GridSearchCV with the RandomForestRegressor and the parameter grid
grid = GridSearchCV(RandomForestRegressor(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='neg_mean_squared_error')

# Fit to the training data
grid.fit(X_train, y_train)

# Make predictions on the testing dataset
y_pred_rf = grid.predict(X_test)

# Computing RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f"RMSE: {rmse_rf}")

# Extract the best estimator
best_rf = grid.best_estimator_

# Get feature importances
feature_importances = best_rf.feature_importances_

# Assuming X_train is a DataFrame, get feature names
feature_names = X_train.columns

# Create a DataFrame to view the feature importances
features_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the DataFrame by the importance of features
features_df.sort_values(by='Importance', ascending=False, inplace=True)

# Print sorted feature importances
print(features_df)
features_df.to_csv("rfjs_coeff_importance.csv", index=False)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
RMSE: 0.4257038436686448
   Feature  Importance
3     SPOS    0.469608
5      ACS    0.211012
1      EES    0.109415
2     JIMS    0.021177
6      NCS    0.020849
18   BIO11    0.020445
4      BFI    0.017852
7     GSOI    0.014718
16  BIO9_1    0.014291
0       TI    0.013632
9     BIO2    0.013516
12    BIO5    0.013181
20   BIO13    0.013154
19   BIO12    0.009077
17   BIO10    0.007650
11    BIO4    0.006592
15    BIO8    0.006270
10    BIO3    0.005191
14    BIO7    0.004568
8     BIO1    0.003979
13    BIO6    0.003822


In [51]:
# Creating a dataset with the model names and their corresponding AUC scores
data = {
    "Model": ["Logistic Regression", "XGBoost", "Neural Network", "SVM", "Random Forest"],
    "RMSE": [rmse_reg, rmse_xgb, rmse_nn, rmse_svr, rmse_rf]
}

df = pd.DataFrame(data)

# Writing the dataset to a .csv file
csv_file_path = "ALL_model_rmse_scores.csv"  # Specify your desired path and filename
df.to_csv(csv_file_path, index=False)